# imports

In [1]:
import requests
import json
import sys
import io
import logging

# settings

In [2]:
st_url = "http://smartaqnet-dev.teco.edu:8080/FROST-Server/v1.0"

# logging

In [3]:
log = logging.getLogger(__name__)
log.setLevel(logging.DEBUG)
formatter = logging.Formatter("%(asctime)s - %(levelname)s(%(name)s): %(message)s")

fh = logging.FileHandler('./log')
fh.setLevel(logging.DEBUG)
fh.setFormatter(formatter)
log.addHandler(fh)

sh = logging.StreamHandler(sys.stderr)
sh.setLevel(logging.ERROR)
sh.setFormatter(formatter)
log.addHandler(sh)

# geo conversion function

In [4]:
def geo_conversion(inputstring):
    direction = {
        "N": 1,
        "S": -1,
        "E": 1,
        "W": -1
    }
    
    new = inputstring.replace(" ", "")
    new = new.replace(u"°", " ")
    new = new.replace("'", " ")
    new = new.replace("\"", " ")
    new = new.split()
    new_dir = new.pop()
    
    return (float(new[0])+float(new[1])/60.0+float(new[2])/3600.0) * direction[new_dir]

# test
geo_conversion("""48°21'28.6"N""")

48.35794444444445

# post entity function

In [5]:
def PostEntity(st_url, where, entity):
        """TODO: add docstring"""
        try:
            entityid = entity["@iot.id"]
        except KeyError:
            entityid = "new ID"
            
        p = requests.post(st_url + where, json.dumps(entity))
        if (p.status_code  == 201):
            log.debug("entity created: " + st_url + where + "('" + entityid + ")'")
            return True
        else:
            body = b"";
            for chunk in p.iter_content(chunk_size=128):
                body += chunk
                
            body = body.decode("utf8")
            body = body.replace("\n", " ").replace("\r", "")
            
            if (p.status_code  == 500 and entityid != "new ID"):
                log.warning("entity exists already: " + st_url + where + "('" + entityid + "')" + " - body: " + body)
                return False
            
            log.error("cannot create entity: " + st_url + where + "('" + entityid + "')" + \
                      " - exit code: " + str(p.status_code) + " - body: " + body)
            raise TypeError("invalid entity")

# define data

In [6]:
exceldata = [
    {
        "name": "Container FH",
        "acronym": "FH",
        "lat": """48°21'28.6"N""",
        "lon": """10°54'25.1"E""",
        "type": "Urban Background",
        "url": "helmholtz-muenchen.de",
        "device": "container",
        "srn": "FH"
    },
    
    {
        "name": "LÜB Königsplatz",
        "acronym": "KP",
        "lat": """48°21'52.5"N""",
        "lon": """10°53'42.1"E""",
        "type": "Traffic Site (City Zone)",
        "url": "lfu.bayern.de",
        "device": "container",
        "srn": "701"
    },
    
    
    {
        "name": "LÜB Bourges Platz",
        "acronym": "BP",
        "lat": """48°22'36.4"N""",
        "lon": """10°53'18.2"E""",
        "type": "Urban Background",
        "url": "lfu.bayern.de",
        "device": "container",
        "srn": "706"
    },
    
    {
        "name": "LÜB Karlsstraße",
        "acronym": "KS",
        "lat": """48°22'13.0"N""",
        "lon": """10°53'46.7"E""",
        "type": "Traffic Site (Hot Spot)",
        "url": "lfu.bayern.de",
        "device": "container",
        "srn": "1401"
    },
    
    {
        "name": "LÜB LfU",
        "acronym": "LfU",
        "lat": """48°19'33.7"N""",
        "lon": """10°54'11.1"E""",
        "type": "Regional Background",
        "url": "lfu.bayern.de",
        "device": "container",
        "srn": "1501"
    },
    
    {
        "name": "Schertlinstraße",
        "acronym": "SST",
        "lat": """48°21'07.2"N""",
        "lon": """10°54'09.9"E""",
        "type": "Traffic Site (Crossing)",
        "url": "helmholtz-muenchen.de",
        "device": "container",
        "srn": "SST"
    },
    
    {
        "name": "Klostergarten",
        "acronym": "KLG",
        "lat": """48°22'27.3"N""",
        "lon": """10°53'59.3"E""",
        "type": "Urban Background",
        "url": "helmholtz-muenchen.de",
        "device": "container",
        "srn": "KLG"
    },
    
    {
        "name": "Lindenau",
        "acronym": "LDH",
        "lat": """48°19'20.6"N""",
        "lon": """10°57'49.6"E""",
        "type": "Rural Background",
        "url": "helmholtz-muenchen.de",
        "device": "container",
        "srn": "LDH"
    }
]

# create things

In [7]:
things = list()

for elem in exceldata:
    thing = dict()
    thing["name"] = "Messcontainer"
    thing["description"] = elem["name"] + " - " + elem["acronym"]
    thing["properties"] = {
        "SiteType": elem["type"]
    }
    thing["@iot.id"] = "saqn:t:" + elem["url"] + ":" + elem["device"] + ":2018:" + elem["srn"]
    elem["thingid"] = thing["@iot.id"]
    
    things.append(thing)

# create locations

In [8]:
locations = list()
for elem in exceldata:
    location = dict()
    location["name"] = "Messcontainer"
    location["description"] = "Koordinaten des Messcontainers"
    location["encodingType"] = "application/vnd.geo+json"
    location["location"] = {
        "type": "Point",
        "coordinates": [
            geo_conversion(elem["lon"]),
            geo_conversion(elem["lat"])
        ]
    }
    
    location["@iot.id"] = "saqn:l:" + elem["url"]
    
    if "Lindenau" in elem["name"]:
        location["@iot.id"] += ":Friedberg:" + elem["name"]
    else:
        location["@iot.id"] += ":Augsburg:" + elem["name"]
        
    elem["locationid"] = location["@iot.id"]
    
    locations.append(location)

# add locations to things

In [9]:
comb = zip(things, locations)
things = list()
for elem in comb:
    elem[0]["Locations"] = [elem[1]]
    things.append(elem[0])

# send everything

In [10]:
for elem in things:
    PostEntity(st_url, "/Things", elem)

# write data

In [11]:
with io.open("exceldata.dat", "w", encoding="utf8") as file:
    json.dump(exceldata, file, ensure_ascii=False)